# Load and Clean Axial Drilling Data
This notebook loads and cleans data from primary sourcing and saves these to JSON and NetCDF files. We also build a logarithmic spiral and use it to place the primary holes.

---
## Load Data

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
xr.set_options(display_style='html');
import shapefile
import geopandas as gpd
from shapely.geometry import Point
import pygmt

In [2]:
data_dir = '../data/'

### Load and Save Vent Data

In [3]:
vents_df = pd.read_excel(data_dir + '/markers-vents-axial-master-updated-post2017.xlsx', sheet_name = 1,
                     usecols=[0, 1, 2, 7, 10], names=['name', 'lat', 'lon', 'type', 'use'])
vents_df.use = vents_df.use.str.casefold()
vents_df = vents_df[vents_df.use == 'yes'].reset_index(drop = True) # drop vents with use=='no'
vents_df.head()

,name,lat,lon,type,use
0,Mkr166,45.933164,-129.982282,diffuse,yes
1,Old Tubeworms,45.933313,-129.982069,diffuse,yes
2,Village,45.926180,-129.980570,diffuse,yes
3,Bag-1,45.916332,-129.989045,snowblower,yes
4,Bag-2,45.917412,-129.988765,snowblower,yes


In [4]:
vents_df.to_json(data_dir + 'vents.json', orient="records", lines=True)

### Define Location of International District Vent Field

In [5]:
international = [['International', 45.9264, -129.9796]]
international_df = pd.DataFrame(international, columns = ['name', 'lat', 'lon'])
international_df

,name,lat,lon
0,International,45.9264,-129.9796


In [6]:
international_df.to_json(data_dir + 'international.json', orient="records", lines=True)

### Load and Save Bathy Data

In [7]:
bathy_grd = 'AxialSummit_MAUV_and_Topo1mSq.grd'

In [8]:
bathy_ds = xr.load_dataset(data_dir + bathy_grd)
bathy_ds = bathy_ds.rename({'z': 'depth'})
bathy_ds

<xarray.Dataset>
Dimensions:       (grid_mapping: 12, lat: 15962, lon: 14815)
Coordinates:
  * lon           (lon) float64 -130.1 -130.1 -130.1 ... -129.9 -129.9 -129.9
  * lat           (lat) float64 45.85 45.85 45.85 45.85 ... 46.06 46.06 46.06
  * grid_mapping  (grid_mapping) |S1 b'' b'' b'' b'' b'' ... b'' b'' b'' b'' b''
Data variables:
    depth         (lat, lon) float32 nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    Conventions:  COARDS, CF-1.5
    title:        Produced by grdmath
    history:      grdmath AxialSummit_2019_Topo1mSq.grd AxialSummit_MAUV_and_...
    description:  \n\tProjection: Geographic\n\tGrid created by mbgrid\n\tMB-...
    GMT_version:  5.4.1 (r18208) [64-bit]

In [9]:
bathy_ds.to_zarr(data_dir + 'bathy.zarr', mode='w')

### Load and Save OOI Infrastructure Data

In [10]:
ooi_df = pd.read_excel(data_dir + 'RSN_Positions_20190906_PUB.xlsx', sheet_name = 2,
                     usecols=[0, 1, 2, 8], names=['name', 'lat', 'lon', 'type'])
boundary_east = -129.8
ooi_df = ooi_df[ooi_df.lon < boundary_east].reset_index(drop = True) # remove infrastructure west of Axial base
ooi_df.head()

,name,lat,lon,type
0,AOABPA301,45.954846,-130.009253,instrument
1,BOTPTA301,45.954850,-130.008772,instrument
2,BOTPTA302,45.939859,-129.974170,instrument
3,BOTPTA303,45.925790,-129.977499,instrument
4,BOTPTA304,45.933638,-130.013897,instrument


In [11]:
ooi_df.to_json(data_dir + 'ooi.json', orient="records", lines=True)

In [12]:
ooi_cables_df = pd.read_excel(data_dir + 'RSN_Positions_20190906_PUB.xlsx', sheet_name = 5,
                     usecols=[1, 2, 3], names=['name', 'lat', 'lon'])
ooi_cables_df = ooi_cables_df[ooi_cables_df.lon < boundary_east].reset_index(drop = True)
ooi_cables_df.head()

,name,lat,lon
0,RS03W2,45.874562,-129.801738
1,RS03W2,45.876163,-129.807545
2,RS03W2,45.883043,-129.829078
3,RS03W2,45.886165,-129.838163
4,RS03W2,45.912712,-129.902963


In [13]:
ooi_cables_df.to_json(data_dir + 'ooi_cables.json', orient="records", lines=True)

### Load and Save Layer 2A Thickness Data

In [14]:
thickness_df = pd.read_csv(data_dir + 'l2A_3D.csv', names=['lon', 'lat', 'thickness'])
thickness_df.thickness = thickness_df.thickness*1000

In [15]:
lat = thickness_df.lat.values.reshape(301, 401).tolist()
lon = thickness_df.lon.values.reshape(301, 401).tolist()
thickness = thickness_df.thickness.values.reshape(301, 401).tolist()

In [16]:
thickness_ds = xr.Dataset({'thickness': (['x', 'y'], thickness)},
                coords = {'lon': (['x', 'y'], lon),
                          'lat': (['x', 'y'], lat)})
thickness_ds

<xarray.Dataset>
Dimensions:    (x: 301, y: 401)
Coordinates:
    lon        (x, y) float64 -130.2 -130.2 -130.2 ... -129.7 -129.7 -129.7
    lat        (x, y) float64 45.77 45.77 45.77 45.78 ... 46.06 46.06 46.06
Dimensions without coordinates: x, y
Data variables:
    thickness  (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan

In [17]:
thickness_ds.to_zarr(data_dir + 'thickness.zarr', mode='w');

### Load and Save Lava Flow Outlines

In [18]:
sf = shapefile.Reader(data_dir + 'Axial-1998-lava-geo.shp')
lava_1998_list = []
for i, record in enumerate(sf.shapeRecords()):
    points = sf.shapeRecords()[i].shape.points
    df = pd.DataFrame(points, columns = ('lon', 'lat'))
    df['area'] = [i]*len(points)
    if i == 1:
        df = df.iloc[0:2710]
    lava_1998_list.append(df)
lava_1998_df = pd.concat(lava_1998_list)

In [19]:
lava_1998_df.to_json(data_dir + 'lava_1998.json', orient="records", lines=True)

In [20]:
sf = shapefile.Reader(data_dir + 'Axial-2011-lava-geo-v1.shp')
lava_2011_list = []
for i, record in enumerate(sf.shapeRecords()):
    points = sf.shapeRecords()[i].shape.points
    df = pd.DataFrame(points, columns = ('lon', 'lat'))
    df['area'] = [i]*len(points)
    if i == 0:
        df = df.iloc[0:454]
    elif i == 1:
        df = df.iloc[0:191]
    elif i == 3:
        df = df.iloc[0:281]
    lava_2011_list.append(df)
lava_2011_df = pd.concat(lava_2011_list)

In [21]:
lava_2011_df.to_json(data_dir + 'lava_2011.json', orient="records", lines=True)

### Load and Save Jason Data

In [22]:
jason_list = [pd.read_json(data_dir + 'rr1712.json', orient="records", lines=True),
              pd.read_json(data_dir + 'rr1713.json', orient="records", lines=True),
              pd.read_json(data_dir + 'rr1714.json', orient="records", lines=True),
              pd.read_json(data_dir + 'rr1715.json', orient="records", lines=True),
              pd.read_json(data_dir + 'km1813.json', orient="records", lines=True),
              pd.read_json(data_dir + 'skq201610s.json', orient="records", lines=True),
              pd.read_json(data_dir + 'tn327.json', orient="records", lines=True),
              pd.read_json(data_dir + 'rb1403.json', orient="records", lines=True),
              pd.read_json(data_dir + 'tn300.json', orient="records", lines=True),
              ]              
jason_df = pd.concat(jason_list)

In [23]:
jason_df = jason_df[jason_df.depth>1400]

In [24]:
jason_df['depth_m'] = pd.Series(jason_df.depth.values.astype(np.int64))

In [25]:
jason_df.to_json(data_dir + 'jason.json', orient="records", lines=True)

---
## Create an Interpolated Rectilinear Thickness Dataset

In [26]:
thickness_interp_ds = pygmt.surface(x=thickness_df.lon.values, y=thickness_df.lat.values, z=thickness_df.thickness.values,
    spacing = '1s', region = '-130.02/-129.94/45.9/45.96')
thickness_interp_ds = thickness_interp_ds.rename({'z': 'thickness'})
thickness_interp_ds

<xarray.Dataset>
Dimensions:    (lat: 217, lon: 289)
Coordinates:
  * lon        (lon) float64 -130.0 -130.0 -130.0 ... -129.9 -129.9 -129.9
  * lat        (lat) float64 45.9 45.9 45.9 45.9 ... 45.96 45.96 45.96 45.96
Data variables:
    thickness  (lat, lon) float32 941.74994 942.1156 ... 1074.003 1080.7026
Attributes:
    Conventions:  CF-1.7
    title:        Data gridded with continuous surface splines in tension
    history:      surface @GMTAPI@-000000 -G/tmp/pygmt-vozbjo_j.nc -I1s -R-13...
    GMT_version:  6.0.0 [64-bit]

In [27]:
thickness_interp_ds.to_zarr(data_dir + 'thickness_interp.zarr', mode='w')

---
## Build Drill Site Database

### Define Preliminary Hole Locations

In [28]:
holes = [['AXIAL_01A', 45.9258, -129.9788],
         ['AXIAL_02A', 45.9244, -129.9760],
         ['AXIAL_03A', 45.9249, -129.9780],
         ['AXIAL_04A', 45.9254, -129.9765],
         ['AXIAL_05A', 45.92003065, -129.96292517]]

holes_df = pd.DataFrame(holes, columns = ['name', 'lat', 'lon'])
holes_df

,name,lat,lon
0,AXIAL_01A,45.925800,-129.978800
1,AXIAL_02A,45.924400,-129.976000
2,AXIAL_03A,45.924900,-129.978000
3,AXIAL_04A,45.925400,-129.976500
4,AXIAL_05A,45.920031,-129.962925


### Build Spiral and Mark Holes

In [29]:
center_y_geo = holes_df[holes_df.name=='AXIAL_01A'].lat.values[0]
center_x_geo = holes_df[holes_df.name=='AXIAL_01A'].lon.values[0]

In [30]:
center_gs_geo = gpd.GeoSeries([Point(center_x_geo, center_y_geo)], crs = 'epsg:4326')
center_gs_utm = center_gs_geo.to_crs('epsg:32609')
center_x_utm = center_gs_utm.to_list()[0].x
center_y_utm = center_gs_utm.to_list()[0].y

In [31]:
theta = np.linspace(0,11 * np.pi, 20000)
#theta = np.linspace(0,13 * np.pi, 23636)
#k = 0.1
k = 0.095
#a = 46
a = 54.55
x = -a*(np.e**(k*theta))*np.cos(theta) + center_x_utm
y = -a*(np.e**(k*theta))*np.sin(theta) + center_y_utm
spiral_df = pd.DataFrame({'x':x, 'y':y})

In [32]:
spiral_gs_utm = gpd.GeoSeries([Point(x,y) for x, y in zip(spiral_df['x'], spiral_df['y'])], crs='EPSG:32609')

In [33]:
spiral_gs_geo = spiral_gs_utm.to_crs('EPSG:4326')
spiral_df_geo = pd.DataFrame(list(zip([point.y for point in spiral_gs_geo], [point.x for point in spiral_gs_geo])), columns = ['lat', 'lon'])

In [34]:
spiral_df_geo.to_json(data_dir + 'spiral.json', orient="records", lines=True)

### Mark Holes on Spiral

In [35]:
holes_new_df = pd.concat([spiral_df_geo.iloc[[8308]], spiral_df_geo.iloc[[12442]], spiral_df_geo.iloc[[15580]]])
holes_new_df['name'] = ['AXIAL_02A', 'AXIAL_03A', 'AXIAL_04A']
holes_df = pd.concat([holes_df[holes_df.name=='AXIAL_01A'], 
                      holes_new_df.reset_index(drop=True), 
                      holes_df[holes_df.name=='AXIAL_05A']])
holes_df

,name,lat,lon
0,AXIAL_01A,45.925800,-129.978800
0,AXIAL_02A,45.923931,-129.978170
1,AXIAL_03A,45.924054,-129.973988
2,AXIAL_04A,45.919632,-129.976727
4,AXIAL_05A,45.920031,-129.962925


In [36]:
holes_df = holes_df.reset_index(drop=True)
holes_df

,name,lat,lon
0,AXIAL_01A,45.925800,-129.978800
1,AXIAL_02A,45.923931,-129.978170
2,AXIAL_03A,45.924054,-129.973988
3,AXIAL_04A,45.919632,-129.976727
4,AXIAL_05A,45.920031,-129.962925


### Add Alternative Holes

In [37]:
alt_holes = [['AXIAL_06A', 45.92198, -129.96827],
         ['AXIAL_07A', 45.91525, -129.97527],
         ['AXIAL_08A', 45.9258, -129.97235],
         ['AXIAL_09A', 45.917954, -129.957211]]

alt_holes_df = pd.DataFrame(alt_holes, columns = ['name', 'lat', 'lon'])
alt_holes_df

,name,lat,lon
0,AXIAL_06A,45.921980,-129.968270
1,AXIAL_07A,45.915250,-129.975270
2,AXIAL_08A,45.925800,-129.972350
3,AXIAL_09A,45.917954,-129.957211


In [38]:
holes_df = pd.concat([holes_df, alt_holes_df]).reset_index(drop = True)
holes_df

,name,lat,lon
0,AXIAL_01A,45.925800,-129.978800
1,AXIAL_02A,45.923931,-129.978170
2,AXIAL_03A,45.924054,-129.973988
3,AXIAL_04A,45.919632,-129.976727
4,AXIAL_05A,45.920031,-129.962925
5,AXIAL_06A,45.921980,-129.968270
6,AXIAL_07A,45.915250,-129.975270
7,AXIAL_08A,45.925800,-129.972350
8,AXIAL_09A,45.917954,-129.957211


### Interpolate Depth and Thickness

In [39]:
holes_df['depth'] = list(np.round(np.diagonal((bathy_ds.depth.interp(lat = holes_df.lat, lon = holes_df.lon)).values)))
holes_df['thickness'] = list(np.round(np.diagonal((thickness_interp_ds.thickness.interp(lat = holes_df.lat, lon = holes_df.lon)).values)))
holes_df

,name,lat,lon,depth,thickness
0,AXIAL_01A,45.925800,-129.978800,-1520.0,703.0
1,AXIAL_02A,45.923931,-129.978170,-1523.0,643.0
2,AXIAL_03A,45.924054,-129.973988,-1530.0,604.0
3,AXIAL_04A,45.919632,-129.976727,-1533.0,560.0
4,AXIAL_05A,45.920031,-129.962925,-1559.0,493.0
5,AXIAL_06A,45.921980,-129.968270,-1543.0,547.0
6,AXIAL_07A,45.915250,-129.975270,-1545.0,577.0
7,AXIAL_08A,45.925800,-129.972350,-1533.0,610.0
8,AXIAL_09A,45.917954,-129.957211,-1579.0,538.0


### Calculate Distance Between Holes

In [40]:
points = []
for i in range(len(holes_df)):
    points.append(Point(holes_df.iloc[i].lon, holes_df.iloc[i].lat))

In [41]:
holes_gs_geo = gpd.GeoSeries(points, crs = 'epsg:4326')
holes_gs_utm = holes_gs_geo.to_crs('epsg:32609')

In [42]:
distances_list = []
for i in range(len(holes_gs_utm)):
    distances_list.append(holes_gs_utm.distance(holes_gs_utm[i]).round().astype(np.int64))

In [43]:
distances_df = pd.DataFrame(distances_list)
distances_df.columns = ['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9']
distances_df

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9
0,0,213,421,704,1388,920,1204,500,1888
1,213,0,325,491,1259,798,990,497,1756
2,421,325,0,535,967,500,983,232,1467
3,704,491,535,0,1071,706,500,765,1525
4,1388,1259,967,1071,0,468,1095,972,500
5,920,798,500,706,468,0,924,529,967
6,1204,990,983,500,1095,924,0,1194,1432
7,500,497,232,765,972,529,1194,0,1462
8,1888,1756,1467,1525,500,967,1432,1462,0


In [44]:
holes_df = pd.concat([holes_df, distances_df], axis=1)
holes_df

,name,lat,lon,depth,thickness,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9
0,AXIAL_01A,45.925800,-129.978800,-1520.0,703.0,0,213,421,704,1388,920,1204,500,1888
1,AXIAL_02A,45.923931,-129.978170,-1523.0,643.0,213,0,325,491,1259,798,990,497,1756
2,AXIAL_03A,45.924054,-129.973988,-1530.0,604.0,421,325,0,535,967,500,983,232,1467
3,AXIAL_04A,45.919632,-129.976727,-1533.0,560.0,704,491,535,0,1071,706,500,765,1525
4,AXIAL_05A,45.920031,-129.962925,-1559.0,493.0,1388,1259,967,1071,0,468,1095,972,500
5,AXIAL_06A,45.921980,-129.968270,-1543.0,547.0,920,798,500,706,468,0,924,529,967
6,AXIAL_07A,45.915250,-129.975270,-1545.0,577.0,1204,990,983,500,1095,924,0,1194,1432
7,AXIAL_08A,45.925800,-129.972350,-1533.0,610.0,500,497,232,765,972,529,1194,0,1462
8,AXIAL_09A,45.917954,-129.957211,-1579.0,538.0,1888,1756,1467,1525,500,967,1432,1462,0


### Save Hole Locations as CSV and JSON

In [45]:
holes_df.to_csv(data_dir + 'holes.csv')

In [46]:
holes_df.to_json(data_dir + 'holes.json', orient="records", lines=True)